In [12]:
from robrats_inference import inference
from utils import make_3_channel_NIFTI
import nibabel as nib
import skimage.measure as measure
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

In [2]:
img_test = nib.load("data/rat20_20240628_T2.nii")

In [3]:
img_test.affine

array([[-0.1171875,  0.       ,  0.       , 15.       ],
       [ 0.       , -0.1171875,  0.       , 15.       ],
       [ 0.       ,  0.       ,  1.       , -8.       ],
       [ 0.       ,  0.       ,  0.       ,  1.       ]])

In [4]:
outputs = inference("data/rat20_20240628_T2.nii", "data/rat20_20240628_ADC.nii", "data/rat20_20240628_DCE.nii", "data/")

T2 shape: (256, 256, 19)
ADC shape: (128, 128, 19)
DCE shape: (128, 128, 19)


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.


In [23]:
volume = nib.load("data/rat20_20240628_T2.nii").get_fdata()
shape = volume.shape

def plot_slice(slice_idx):
    """Plot a 2D slice from a 3D NIFTI image."""
    plt.figure(figsize=(6,6))


    

    plt.imshow(volume[:,:,slice_idx], cmap="gray")

    colors = ["red", "yellow", "green", "blue"]


    for i, color in enumerate(colors):
        out = outputs[0][i, :, :, slice_idx].detach().cpu().numpy()

        contours = measure.find_contours(out, level=0.5)

        for contour in contours:

            plt.plot(contour[:, 1], contour[:, 0], color=color, linewidth=1.5)

    plt.axis('off')
    plt.show()


In [24]:
slice_slider = widgets.IntSlider(
    min=0, max=shape[2]-1, step=1, value=shape[2]//2,
    description='Slice:',
    style={'description_width': 'initial'}
)



interact(plot_slice, slice_idx=slice_slider)

interactive(children=(IntSlider(value=9, description='Slice:', max=18, style=SliderStyle(description_width='in…

<function __main__.plot_slice(slice_idx)>